## A quick trial.

In [ ]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [ ]:
from transformers import AutoProcessor, MusicgenForConditionalGeneration

processor = AutoProcessor.from_pretrained("facebook/musicgen-small")
model = MusicgenForConditionalGeneration.from_pretrained("facebook/musicgen-small")
model = model.cuda()

## Text-conditional Generation

In [ ]:
import json

text=[]
text.append("country guitar")

# From the Stanford captioning dataset. This alone have high probability will generate terrible music.
text.append("A large building with bars on the windows in front of it. \
            There is people walking in front of the building. \
            There is a street in front of the building with many cars on it.")

inputs = processor(
    text=text,
    padding=True,
    return_tensors="pt",
).to("cuda")


In [ ]:
# man_new_tokens = 256/512/1024 for 5/10/20 seconds
audio_values = model.generate(**inputs, do_sample=True, guidance_scale=3, max_new_tokens=256)

Can either play it in ipynb or save it.

In [ ]:
import IPython
sampling_rate = model.config.audio_encoder.sampling_rate

IPython.display.Audio(audio_values[0].cpu().numpy(), rate=sampling_rate)



In [ ]:
IPython.display.Audio(audio_values[1].cpu().numpy(), rate=sampling_rate)

In [ ]:
# The dimesions are (batch_size, num_channels, sequence_length))
print(audio_values.shape)

In [ ]:
import scipy, os
output_dir = "./audio_out"
os.makedirs(output_dir, exist_ok=True)
sampling_rate = model.config.audio_encoder.sampling_rate
for i in range(len(audio_values)):
    scipy.io.wavfile.write(f"{output_dir}/musicgen_out_{i}.wav", rate=sampling_rate, data=audio_values[i, 0].numpy())